In [ ]:
# only for google drive
!pip install git+https://github.com/seankmartin/simuran.git@main
!pip install allensdk==2.14.1
!pip install one-api==1.17.0
!pip install ibllib==2.19.0

# Currently need allensdk 2.13.5 to work with manifest 0.3.0 but that should change in the future

In [42]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [82]:
from pathlib import Path
from pprint import pprint
import simuran
from simuran.loaders.allen_loader import AllenVisualBehaviorLoader
from simuran.loaders.one_loader import OneAlyxLoader

In [44]:
ALLEN_CACHE_DIR = Path(r"D:\example-data\allen-data")
ALLEN_MANIFEST = "visual-behavior-neuropixels_project_manifest_v0.4.0.json"
IBL_CACHE_DIR = Path(r"D:\example-data\ibl-data")

In [ ]:
# loader = AllenVisualBehaviorLoader(cache_directory=ALLEN_CACHE_DIR, manifest=ALLEN_MANIFEST)
# loader.create_s3_cache()
# loader.cache.compare_manifests(ALLEN_MANIFEST, ALLEN_MANIFEST.replace("3", "4"))

In [ ]:
def create_allen_recording_container(cache_directory, manifest=None) -> "simuran.RecordingContainer":
    loader = AllenVisualBehaviorLoader(cache_directory=cache_directory, manifest=manifest)
    loader.create_s3_cache()
    ecephys_sessions = loader.cache.get_ecephys_session_table()
    sessions = ecephys_sessions.dropna(subset=["file_id"])
    recording_container = simuran.RecordingContainer.from_table(ecephys_sessions, loader)
    return recording_container

In [ ]:
allen_rc = create_allen_recording_container(ALLEN_CACHE_DIR, ALLEN_MANIFEST)
for r in allen_rc:
    if r.attrs["downloaded"]:
        print(f"{r.source_file} is downloaded")
        r.load()
        print(r)
    else:
        print(f"{r.source_file} is not yet available")

In [80]:
def create_one_alyx_recording_container(cache_directory):
    loader = OneAlyxLoader.from_cache(cache_directory)
    eid = loader.find_eid("steinmetzlab", "NR_0017")[0][0]
    rc = simuran.RecordingContainer()
    rc.append(simuran.Recording(attrs={"experiment_id": eid}, loader=loader))
    rc.loader = loader 
    return rc

In [85]:
ibl_rc = create_one_alyx_recording_container(IBL_CACHE_DIR)
for r in ibl_rc.load_iter():
    with open("output.txt", "w") as out:
        pprint(r.data, stream=out)